![image-3.png](https://github.com/cafawo/Derivatives/blob/main/figures/fscampus_small.png?raw=1)

# Derivatives

[**Prof. Dr. Fabian Woebbeking**](https://woebbeking.info/)</br>
Assistant Professor of Financial Economics</br>

## Case 6: Credit pricing and Credit Default Swaps (CDS)

### Part 1

#### Task: Price a 8% 3-year B rated bond using risk adjusted discounting.

**Spot rates** (percentage values, rounded to two digits):

| **Rating** | **1Y**       | **2Y**       | **3Y**       | **4Y**       | **5Y**       |
|------------|--------------|--------------|--------------|--------------|--------------|
|     AAA    |     3.65     |     3.86     |     4.30    |     4.63    |     4.98    |
|     AA     |     3.70    |     3.90    |     4.34    |     4.68    |     5.03    |
|     A      |     3.77    |     3.98    |     4.43    |     4.77    |     5.12    |
|     BBB    |     3.89    |     4.12    |     4.58    |     4.92    |     5.28    |
|     BB     |     4.32    |     4.64    |     5.17    |     5.58    |     6.00    |
|     B      |     4.93    |     5.34    |     5.98    |     6.46    |     6.96    |
|     CCC    |     5.84    |     6.44    |     7.26    |     7.84    |     8.44    |

In [25]:
import numpy as np

spot_rates = np.array([4.93, 5.34, 5.98]) / 100
discount_factors = 1 / (1 + spot_rates)**np.array([1, 2, 3])
annuity_factor = sum(discount_factors)
price_discounting = 8 * annuity_factor + 100 / (1 + spot_rates[-1])**3

print(f"The bond price is EUR {price_discounting:,.2f}")

The bond price is EUR 105.56


#### Task: Price a 8% 3-year B rated junior bond using risk adjusted cash flows.

**Spot rates** (percentage values, rounded to two digits):

|           | **1Y** | **2Y**  | **3Y**  | **4Y**  | **5Y**  |
|-----------|--------|---------|---------|---------|---------|
| Spot rate | 3.60   | 3.81    | 4.25    | 4.58    | 4.93    |


**Recovery rates** (percentage values, rounded to two digits):

| **Seniority**     | **Recovery** |
|-------------------|--------------|
| Senior secured    | 53.08        |
| Senior            | 51.13        |
| Junior            | 32.74        |


**Cumulative default rates** (percentage values, rounded to two digits):

| **Rating** | **1Y**       | **2Y**       | **3Y**       | **4Y**       | **5Y**      | **7Y**    | **10Y**   | **15Y**      |
|------------|--------------|--------------|--------------|--------------|-------------|-----------|-----------|--------------|
|     AAA    |     0.00     |     0.00     |     0.07     |     0.15     |     0.24    |     0.66  |     1.40  |     1.40     |
|     AA     |     0.00     |     0.02     |     0.12     |     0.25     |     0.43    |     0.89  |     1.29  |     1.48     |
|     A      |     0.06     |     0.16     |     0.27     |     0.44     |     0.67    |     1.12  |     2.17  |     3.00     |
|     BBB    |     0.18     |     0.44     |     0.72     |     1.27     |     1.78    |     2.99  |     4.34  |     4.70     |
|     BB     |     1.06     |     3.48     |     6.12     |     8.68     |     10.97   |     14.46 |     17.73 |     19.91    |
|     B      |     5.20     |     11.00    |     15.95    |     19.40    |     21.88   |     25.14 |     29.02 |     30.65    |
|     CCC    |     19.79    |     26.92    |     31.63    |     35.97    |     40.15   |     42.64 |     45.10 |     45.10    |

The PV of the bond using risk adjusted cash flows, i.e. cash flows adjusted for the probability of their occurrence, is

$$
\text{PV Bond} = \sum_{t=1}^T \frac{C_t \times (1-p_t) + C_t \times  \text{Recovery} \times p_t}{(1 + r_t)^t},
$$

where $p_t$ is the cumulative probability up until time $t$.

In [26]:
import numpy as np

cash_flows = np.array([8, 8, 108])
spot_rates = np.array([3.6, 3.81, 4.25]) / 100
default_probabilities = np.array([5.2, 11, 15.95]) / 100

pv = 0
t  = 1
for c, p, r in zip(cash_flows, default_probabilities, spot_rates):
    cf_t = c * (1-p) + c * p * 0.3274
    pv += cf_t / (1 + r)**t
    t += 1

print(f"The bond price is EUR {pv:,.2f}")

The bond price is EUR 99.42


#### Task: Calculate the assets swap level.

Use the given swap quotation (Reuters: ICAPEURO) to derive the Asset Swap Level for a 5y-coupon bond with $3$% coupon that is currently quoted at $101.00$

![image-3.png](6_credit_swapquotes.png)

In [27]:
from scipy.optimize import minimize

def YTM(price, coupon, maturity):
    # calculate PV from irr
    def YTM_price(irr, coupon, maturity):
        T = maturity
        annuity_factor = (1 - (1 + irr) ** -T) / irr
        p = coupon * annuity_factor + 1 / (1 + irr) ** T
        return p
    # target function to be minimized
    def NPV2(irr):
        return (price - YTM_price(irr, coupon, maturity)) ** 2
    # set NPV = 0 and return both PVs and the irr
    opt = minimize(NPV2, x0=0.01, bounds = ((None, None),))
    return opt.x[0]


asset_swap_level = YTM(101/100, 3/100, 5) - 1.528/100

print(f"The asset swap level is Euribor {asset_swap_level*10000:.0f} bps")

The asset swap level is Euribor 125 bps


### Part 2

#### Task: Credit Default Swaps on the company „Unhappy“ are to be priced. 

The settlement in case of a credit event is „Defaulted Bond against Par“. To derive the CDS price, several Unhappy bonds are given. If not stated differently, all bonds currently trade at $100$. 

1. Price a 3y-CDS on Unhappy, use a 3y-Unhappy Floater with a Libor Spread of $1.68$%. How does the hedge for this CDS work?
2. Price a 3y-CDS on Unhappy, use a 3y Unhappy-Floater with a spread of $3.46$% currently trading at $105$.

The current interest rate environment is: 1Y $3.65$%; 2Y $3.86$%; 3Y $4.30$% (ISMA)

In [28]:
# 1.

print(f"The CDS spread is approx. {1.68}%")

# Hedge:
## Sell the risky par floater
## Buy the risk-free par floater

The CDS spread is approx. 1.68%


In [29]:
# 2.

spot_rates = np.array([3.65, 3.86, 4.30]) / 100
discount_factors = 1 / (1 + spot_rates)**np.array([1, 2, 3])
annuity_factor = sum(discount_factors)

fair_spread = 3.46 + (100 - 105) / annuity_factor

print(f"The fair CDS spread is approx. {fair_spread:.2f}%")

The fair CDS spread is approx. 1.66%


#### Task: Calculate the CDS spread from the asset swap level of a 3y-bond currently trading at $102$, with a coupon of $7$%.

The current interest rate environment is: 1Y $3.65$%; 2Y $3.86$%; 3Y $4.30$% (ISMA)

In [30]:
spot_rates = np.array([3.65, 3.86, 4.30]) / 100
discount_factors = 1 / (1 + spot_rates)**np.array([1, 2, 3])
annuity_factor = sum(discount_factors)

fair_spread = (discount_factors[-1] + 7/100 * annuity_factor - 102/100) / annuity_factor

print(f"The fair CDS spread is approx. {fair_spread*100:.2f}")


The fair CDS spread is approx. 2.00


### Part 3

#### Task: Find the constant hazard rates.

Find the constant hazard rates using a par 2y-CDS with a spread of $1.48$%. Risk-free spot rates of $3.65$% for 1y and $3.86$% for 2y (ISMA). The recovery rate is $30$%. Using the hazard rates show that the CDS spread is $1.48$%.

Hint: If you did not get a solution for $h$, you can use $h = 0.0209224$

In [31]:
import numpy as np

def CDS_AB(h, r, T):
    # assuming a constant hazard rate h
    A_T = 0
    B_T = 0
    for t in range(1, T+1):
        r_t = r[t-1] if isinstance(r, (list, np.ndarray)) else r
        A_T += np.exp(-(h + r_t)*t)
        B_T += np.exp(-r_t*t) * ( np.exp(-h*(t-1)) - np.exp(-h*t) )

    return (A_T, B_T)

# Target function for the solver
def fun(h):
    A_T, B_T = CDS_AB(h, r, T)
    return (B_T*(1 - R) / A_T - S_mkt)**2*100

# Parameters
T = 2
R = 30/100
S_mkt = 1.48/100

# Credit triangle
first_guess = S_mkt / (1 - R)
# Find h that solves for S_mkt
h = minimize(fun, x0=first_guess, bounds = ((0, None),)).x[0]
print(f"The calibrated hazard rate is h = {h:.8f}")

# Use h to calculate the CDS spread
A_T, B_T = CDS_AB(h, np.log([1.0365, 1.0386]), 2)
S = B_T*(1 - R) / A_T
print(f"The fair CDS spread is S = {S*100:.2f}%")



The calibrated hazard rate is h = 0.02092239
The fair CDS spread is S = 1.48%


#### Task: Using the data of the question before, find the upfront payment on EUR $10$ million notional for a CDS standard spread of $1$%.

Hint: If you did not get a solution for $h$, you can use $h = 0.0209224$

In [32]:
# Upfront payment (spread payment of 100 basis points = 0.01)
upfront = (B_T*(1 - R) - A_T * 0.01) *10**7 
upfront_alt = (S - 0.01) * A_T *10**7

print(f"Upfront payment on 10mln nominal is EUR {upfront:,.2f}")
print(f"Upfront payment on 10mln nominal is EUR {upfront_alt:,.2f}")

Upfront payment on 10mln nominal is EUR 88,024.89
Upfront payment on 10mln nominal is EUR 88,024.89


In [38]:
print(f"Upfront payment = {(S - 0.01) * A_T * 100:.7f}%")

Upfront payment = 0.8802489%
